In [2]:
!pip install tensorflow_text
!pip install tensorflow
!pip install gpl
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 74.2 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 kB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 4.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 97.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 115.6 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 82.6 MB/s eta 0:00:00
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.12:
      Successfully uninstalled flatbuffers-1.12
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Succe

In [3]:
import pandas as pd 
import re 
import json

from tqdm.autonotebook import tqdm
import gpl
from sentence_transformers import SentenceTransformer

2023-02-24 13:48:26.087016: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-24 13:48:27.015374: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.6/lib64
2023-02-24 13:48:27.015503: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.6/lib64
2023-02-24 13:48:27.015511: W tensorflow/compiler/tf2tensorrt/utils/py_utils

In [4]:
# functions to perform pre-processing
def preprocess(text):
    text = re.sub(r"[^a-zA-Z0-9,.'?]+", ' ', str(text))
    text = [text[:len(text)//2],text[len(text)//2:]]
    return text

def split_first(text):
    return len(text.split())

def convert_str(text):
    return str(text)+"_"


In [ ]:
df = df.apply(lambda x:x.str.lower())

df = df.dropna()

df['new_text']= df.Resume.apply(preprocess)

df = df.explode("new_text")

df = df.reset_index(drop=True)

df["_id"] = df.index

df['num']=df['new_text'].apply(split_first)

df = df[df['num']<400]

df["_id"] = df["_id"].apply(convert_str)

df['title'] = ""

df['metadata'] = ""

df['title'] = df['title'].astype(str)

df['text'] = df['new_text'].astype(str)

df['_id'] = df['_id'].astype(str)

df['concat'] = "qgen" + df["title"] + " " + df["text"]

df[['_id', 'title', 'text', 'metadata']].to_json('corpus.json',orient='records')

df[['_id', 'title', 'text', 'metadata']].to_json('corpus.jsonl',orient='records',lines=True)



f = open('corpus.json')
  
# returns JSON object as 
# a dictionary
data = json.load(f)

In [ ]:
# write to file
with open('corpus.jsonl', 'w') as outfile:
    for entry in data:
        json.dump(entry, outfile)
        outfile.write('\n')

In [ ]:
# dump the correct format
filepath = 'corpus.jsonl'


with open(filepath, 'r') as infile, open('output.json', 'w') as outfile:
    data = [json.loads(line) for line in infile]
    json.dump(data, outfile)

In [ ]:
## convert to csv
df = pd.read_json('output.json')

df['num']=df['text'].apply(split_first)

df = df[df['num']<400]

df.to_csv('final_.csv')



In [ ]:
# read the dataset
df = pd.read_csv('final_.csv')

In [ ]:
# Query Generation via T5
from transformers import T5Tokenizer, T5ForConditionalGeneration
model_name = 'doc2query/msmarco-t5-base-v1'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

## load the correct model class like t5forseq2seq or conditional?
print(tokenizer.__class__.__name__)
print(model.__class__.__name__)

passage = df['text'].iloc[5]

# tokenize the passage
inputs = tokenizer(passage, return_tensors='pt')
# generate three queries
outputs = model.generate(
    input_ids=inputs['input_ids'],
    attention_mask=inputs['attention_mask'],
    max_length=64,
    do_sample=True,
    top_p=0.95,
    num_return_sequences=3
)

print("Paragraph:")
print(passage)

print("\nGenerated Queries:")
for i in range(len(outputs)):
    query = tokenizer.decode(outputs[i], skip_special_tokens=True)
    print(f'{i + 1}: {query}')

In [ ]:
from tqdm.auto import tqdm  # this is our progress bar

target = 200_000
batch_size = 128 #256
num_queries = 3  # number of queries to generate for each passage
count = 0
lines = []
passage_batch = []

# reinitialize passage generator
passages = df['text'].to_numpy() #get_text()

with tqdm(total=target) as progress:
    for passage in passages:
        if count >= target: break
        # remove tab + newline characters if present
        passage_batch.append(passage.replace('\t', ' ').replace('\n', ' '))
        
        # we encode in batches
        if len(passage_batch) == batch_size:
            # tokenize the passage
            inputs = tokenizer(
                passage_batch,
                truncation=True,
                padding=True,
                max_length=256,
                return_tensors='pt'
            )

            # generate three queries per doc/passage
            outputs = model.generate(
                input_ids=inputs['input_ids'].cuda(),
                attention_mask=inputs['attention_mask'].cuda(),
                max_length=64,
                do_sample=True,
                top_p=0.95,
                num_return_sequences=num_queries
            )

            # decode query to human readable text
            decoded_output = tokenizer.batch_decode(
                outputs,
                skip_special_tokens=True
            )

            # loop through to pair query and passages
            for i, query in enumerate(decoded_output):
                query = query.replace('\t', ' ').replace('\n', ' ')  # remove newline + tabs
                passage_idx = int(i/num_queries)  # get index of passage to match query
                lines.append(query+'\t'+passage_batch[passage_idx])
                count += 1
            
            passage_batch = []
            progress.update(len(decoded_output))

# write (Q, P+) pairs to file
with open('pairs.tsv', 'w', encoding='utf-8') as fp:
    fp.write('\n'.join(lines))

In [ ]:
with open('pairs.tsv', 'w', encoding='utf-8') as fp:
    fp.write('\n'.join(lines))


In [ ]:
#negative sampling
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('msmarco-distilbert-base-tas-b')
model.max_seq_length = 256

from tqdm.auto import tqdm

def get_text():
    with open('pairs.tsv', 'r', encoding='utf-8') as fp:
        lines = fp.read().split('\n')
    for line in tqdm(lines):
        try:
            query, passage = line.split('\t')
            yield query, passage
        except ValueError:
            # in case of malformed data, pass onto next row
            pass

In [ ]:

pair_gen = get_text()

for i, (query, passage) in enumerate(pair_gen):
    print(query)
    print()
    print(passage)
    break

In [ ]:
# save embeddings 

! pip install pinecone-client

import pinecone  # pip install pinecone-client

API_KEY = ""  # get api key app.pinecone.io

pinecone.init(
    api_key=API_KEY,
    environment="us-east1-gcp"  # find next to API key in console
)
# create a new negative mining index if does not already exist
if 'negative-mine' not in pinecone.list_indexes():
    pinecone.create_index(
        'negative-mine',
        dimension=model.get_sentence_embedding_dimension(),
        metric='dotproduct',
        pods=1
    )
# connect
index = pinecone.Index('negative-mine')

pinecone.describe_index("negative-mine")



pair_gen = get_text()

pairs = []
to_upsert = []
passage_batch = []
id_batch = []
batch_size = 64

for i, (query, passage) in enumerate(pair_gen):
    pairs.append((query, passage))
    # we do this to avoid passage duplication in the vector DB
    if passage not in passage_batch: 
        passage_batch.append(passage)
        id_batch.append(str(i))
    # on reaching batch_size, we encode and upsert
    if len(passage_batch) == batch_size:
        embeds = model.encode(passage_batch).tolist()
        # upload to index
        index.upsert(vectors=list(zip(id_batch, embeds)))
        # refresh batches
        passage_batch = []
        id_batch = []
        
# check number of vectors in the index
index.describe_index_stats()

In [ ]:
# join the dataset


import random

batch_size = 100
triplets = []

for i in tqdm(range(0, len(pairs), batch_size)):
    # embed queries and query pinecone in batches to minimize network latency
    i_end = min(i+batch_size, len(pairs))
    queries = [pair[0] for pair in pairs[i:i_end]] #100
    pos_passages = [pair[1] for pair in pairs[i:i_end]] #100
    # create query embeddings
    query_embs = model.encode(queries, convert_to_tensor=True, show_progress_bar=False) #100*764
    # search for top_k most similar passages

    res = dict()
    res['results'] = [index.query(i.tolist(), top_k=10) for i in query_embs]

    
    # iterate through queries and find negatives
    for query, pos_passage, query_res in zip(queries, pos_passages, res['results']):
        top_results = query_res['matches']
        # shuffle results so they are in random order
        random.shuffle(top_results)
        for hit in top_results:
            neg_passage = pairs[int(hit['id'])][1]
            # check that we're not just returning the positive passage
            if neg_passage != pos_passage:
                # if not we can add this to our (Q, P+, P-) triplets
                triplets.append(query+'\t'+pos_passage+'\t'+neg_passage)
                break

with open('triplets.tsv', 'w', encoding='utf-8') as fp:
    fp.write('\n'.join(triplets))

In [ ]:
!head -n 2 triplets.tsv

In [ ]:
# score the negative and positive examples

from sentence_transformers import CrossEncoder

model = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
model

In [ ]:
from tqdm.auto import tqdm

def get_lines():
    # loop through each file
    with open('triplets.tsv', 'r', encoding='utf-8') as fp:
        lines = fp.read().split('\n')
    # loop through each line in the current file
    for line in tqdm(lines):
        q, p, n = line.split('\t')
        # return the query, positive, negative
        yield q, p, n
        
lines = get_lines()
label_lines = []

for line in lines:
    q, p, n = line
    p_score = model.predict((q, p))
    n_score = model.predict((q, n))
    margin = p_score - n_score
    # append pairs to label_lines with margin score
    label_lines.append(
        q + '\t' + p + '\t' + n + '\t' + str(margin)
    )

with open("triplets_margin.tsv", 'w', encoding='utf-8') as fp:
    fp.write('\n'.join(label_lines))

In [ ]:
from tqdm.auto import tqdm
from sentence_transformers import InputExample

training_data = []

with open('triplets_margin.tsv', 'r', encoding='utf-8') as fp:
    lines = fp.read().split('\n')
# loop through each line and return InputExample
for line in tqdm(lines):
    q, p, n, margin = line.split('\t')
    training_data.append(InputExample(
        texts=[q, p, n],
        label=float(margin)
    ))

len(training_data)

In [ ]:
# training 

import torch

torch.cuda.empty_cache()

batch_size = 32

loader = torch.utils.data.DataLoader(
    training_data, batch_size=batch_size, shuffle=True
)

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('msmarco-distilbert-base-tas-b')
model.max_seq_length = 256
model

In [ ]:
from sentence_transformers import losses

loss = losses.MarginMSELoss(model)


In [ ]:
epochs = 10
warmup_steps = int(len(loader) * epochs * 0.1)

model.fit(
    train_objectives=[(loader, loss)],
    epochs=epochs,
    warmup_steps=warmup_steps,
    output_path='msmarco-distilbert-base-tas-b-final',
    show_progress_bar=True
)

In [8]:

from sentence_transformers import SentenceTransformer
model = SentenceTransformer('msmarco-distilbert-base-tas-b-final')

[2023-02-24 14:17:43] INFO [sentence_transformers.SentenceTransformer.__init__:66] Load pretrained SentenceTransformer: msmarco-distilbert-base-tas-b-final
[2023-02-24 14:17:44] INFO [sentence_transformers.SentenceTransformer.__init__:105] Use pytorch device: cuda


### query

### document

In [15]:
queries = ["""I have 3.5+ years of work experience and was working as a data scientist with 3 different organizations. I was responsible for using predictive modelling, data processing, and data mining algorithms to solve challenging business problems.
My technology stack includes but not limited to, are python, machine learning, deep learning, time-series, web scraping, flask, FastAPI, snowflake SQL servers, deploying production based servers, keras, TensorFlow, hugging face, Big Data and Data Warehouses. In my career, my growth has been exponential, and I developed interpersonal skills, now I know how to handle a project end to end.
My area of interests are applied machine learning, deep neural network, time series and everything around NLP in the field of ecommerce and consumer internet. My research focus is on information retrieval involving neuroscience and deep reinforcement learning.
I like to listen to a lot of learning courses and read research papers involving deep learning. In my spare time I like to keep up with the news, read blogs on medium and watch a few sci-fi films.""",
           "Snehil started his entrepreneurial journey 14 years ago with the launch of a social networking site along with music and video streaming portals back in 2006, while he was still in school. In 2011 while pursuing engineering in Computer Science, he joined Letsbuy, an e-commerce startup, where he developed and launched their mobile app and site while mobile-commerce was still in its nascent stage in India. Letsbuy was later acquired by Flipkart in 2012.Snehil also co-founded Findyahan, a services marketplace, which was eventually acquired in 2016 by Zimmber. Snehil joined Zimmber as Vice President of Product & Marketing. Zimmber was later acquired by Quikr.",
           """I have over 7 years of combined experience in the fields of data science and machine learning. I've led many data science projects in a wide array of industries. I mainly program in Python using its popular data science libraries.For deep learning, my go to framework is PyTorch. I’ve also worked a significant amount with relational databases and cloud environments.
Worked on diverse array of projects where I used my machine learning expertise to build and advise external clients on how to move forward with machine learning projects. I also advised on how to best collect and structure data.
Other than work, I write a significant amount with regards to AI. I’ve published several deep learning tutorials,focusing on the PyTorch framework. My articles are published on Medium under the publication A Coder’s Guide to AI."""]
           
queries

['I have 3.5+ years of work experience and was working as a data scientist with 3 different organizations. I was responsible for using predictive modelling, data processing, and data mining algorithms to solve challenging business problems.\nMy technology stack includes but not limited to, are python, machine learning, deep learning, time-series, web scraping, flask, FastAPI, snowflake SQL servers, deploying production based servers, keras, TensorFlow, hugging face, Big Data and Data Warehouses. In my career, my growth has been exponential, and I developed interpersonal skills, now I know how to handle a project end to end.\nMy area of interests are applied machine learning, deep neural network, time series and everything around NLP in the field of ecommerce and consumer internet. My research focus is on information retrieval involving neuroscience and deep reinforcement learning.\nI like to listen to a lot of learning courses and read research papers involving deep learning. In my spa

In [17]:
document = """B.Tech / M.Tech degree in Computer Science from a premiere institute.
Should have 1 - 5 years of experience in designing, developing and deploying software, preferably Statistical and Machine Learning models.
Ability to work independently with strong problem solving skills.
Should have excellent knowledge in fundamentals of Machine Learning and Artificial Intelligence, especially in Regression, Forecasting and Optimization.
Should have excellent foundational knowledge in Probability, Statistics and Operations Research/Optimization techniques.
Should have hands on experience thorugh ML Lifecycle from EDA to model deployment.
Should have hands on experience data analysis tools like Jupyter, and packages like Numpy, Pandas, Matplotlib.
Should be hands-on in writing code that is reliable, maintainable, secure, performance optimized.
Should have good knowledge in Cloud Platforms and Service oriented architecture and design"""

document

'B.Tech / M.Tech degree in Computer Science from a premiere institute.\nShould have 1 - 5 years of experience in designing, developing and deploying software, preferably Statistical and Machine Learning models.\nAbility to work independently with strong problem solving skills.\nShould have excellent knowledge in fundamentals of Machine Learning and Artificial Intelligence, especially in Regression, Forecasting and Optimization.\nShould have excellent foundational knowledge in Probability, Statistics and Operations Research/Optimization techniques.\nShould have hands on experience thorugh ML Lifecycle from EDA to model deployment.\nShould have hands on experience data analysis tools like Jupyter, and packages like Numpy, Pandas, Matplotlib.\nShould be hands-on in writing code that is reliable, maintainable, secure, performance optimized.\nShould have good knowledge in Cloud Platforms and Service oriented architecture and design'

In [18]:
from sentence_transformers import util
def score_cos_sim(art1,art2):
    scores = util.cos_sim(art1, art2)[0]
    return scores

In [27]:
def score_inference(queries,document,model):
    
    score = dict()
    
    queries_encode = [model.encode(text) for text in queries]
    document_encode = model.encode(document)
    
    for i,query in enumerate(queries_encode):
        score["document_"+str(i)] = score_cos_sim(query,document_encode)
    return score

In [28]:
score_inference(queries,document,model)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'document_0': tensor([0.8477]),
 'document_1': tensor([0.7088]),
 'document_2': tensor([0.8285])}